In [6]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

In [7]:
df = pd.read_csv('dataset.csv')

In [8]:
df.head()

,label,text
0,1,"#1 Sa aming oferta, makakatanggap ka ng 100% l..."
1,1,"Mga kasamahan, ang iyong 777 ay maaari nang i-..."
2,1,Congratulations! Ikaw ay isa sa mga napiling 9...
3,1,Gusto mo bang kumita ng 100% pa? Huwag palampa...
4,1,Eksklusibong alok para sa iyo: Libreng iPhone ...


In [9]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [10]:
df['label'].value_counts(normalize = True)

label
0    0.534188
1    0.465812
Name: proportion, dtype: float64

In [11]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [14]:
model_name = "jcblaise/bert-tagalog-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)


Some weights of the model checkpoint at jcblaise/bert-tagalog-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
class SpamDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [16]:
train_dataset = SpamDataset(train_encodings, train_df['label'])
test_dataset = SpamDataset(test_encodings, test_df['label'])

NameError: name 'train_encodings' is not defined

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    output_dir='./results',
    evaluation_strategy="steps",
    num_train_epochs=3,
    save_total_limit=2,
    save_steps=500,
    eval_steps=500,
    logging_dir='./logs',
    logging_steps=10,
    do_train=True,
    do_eval=True,
    load_best_model_at_end=True,
)

In [ ]:
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'precision': precision_score(p.label_ids, preds),
        'recall': recall_score(p.label_ids, preds),
        'f1': f1_score(p.label_ids, preds),
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train()
results = trainer.evaluate(test_dataset)